# MOBPY Example: Insurance Charges Analysis
This notebook demonstrates monotonic optimal binning on the Insurance dataset with a continuous target variable (charges).

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import MOBPY components
from MOBPY import MonotonicBinner, BinningConstraints
from MOBPY.plot import (
    plot_bin_statistics, plot_pava_comparison,
    plot_event_rate, plot_sample_distribution,
    plot_bin_boundaries, plot_csd, plot_gcm
)
from MOBPY.core import MergeStrategy

# Set style and random seed
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

##### 1. Load and Explore the Insurance Dataset

In [ ]:
# Load the insurance dataset
df = pd.read_csv('/Users/chentahung/Desktop/git/mob-py/data/insurance3r2.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumn information:")
print(df.dtypes)
print(f"\nTarget (charges) statistics:")
print(df['charges'].describe())

In [ ]:
# Visualize the target distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Histogram
ax1.hist(df['charges'], bins=50, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Charges')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Insurance Charges')

# Log-scale histogram
ax2.hist(np.log1p(df['charges']), bins=50, edgecolor='black', alpha=0.7, color='green')
ax2.set_xlabel('Log(Charges + 1)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Log-Transformed Charges')

plt.tight_layout()
plt.show()

##### 2. Feature Relationships with Target

In [ ]:
# Examine relationships between features and charges
numeric_features = ['age', 'bmi', 'steps', 'children']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, feature in enumerate(numeric_features):
    ax = axes[idx]
    
    # Scatter plot with trend line
    ax.scatter(df[feature], df['charges'], alpha=0.5, s=20)
    
    # Add trend line
    z = np.polyfit(df[feature], df['charges'], 1)
    p = np.poly1d(z)
    ax.plot(sorted(df[feature]), p(sorted(df[feature])), "r--", linewidth=2)
    
    ax.set_xlabel(feature.capitalize())
    ax.set_ylabel('Charges')
    ax.set_title(f'{feature.capitalize()} vs Insurance Charges')
    
    # Add correlation
    corr = df[feature].corr(df['charges'])
    ax.text(0.05, 0.95, f'Corr: {corr:.3f}', transform=ax.transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

##### 3. Monotonic Binning: BMI Example

In [ ]:
# Create constraints for continuous target
constraints = BinningConstraints(
    max_bins=6,
    min_bins=3,
    min_samples=0.05,      # At least 5% of data per bin
    max_samples=0.40,      # No more than 40% in any bin
    initial_pvalue=0.4,
    maximize_bins=False
)

# Fit the binner
bmi_binner = MonotonicBinner(
    df=df,
    x='bmi',
    y='charges',
    constraints=constraints,
    sign='auto',
    strict=True
)

bmi_binner.fit()

print(f"Monotonicity direction: {bmi_binner.resolved_sign_}")
print(f"Number of bins created: {len(bmi_binner.bins_())}")

##### 3.1 Understanding PAVA Process

In [ ]:
# Visualize the PAVA algorithm steps
fig = plot_pava_comparison(bmi_binner, figsize=(14, 6))
plt.show()

# Get detailed PAVA information
pava_groups = bmi_binner.pava_groups_()
print(f"\nNumber of unique BMI values: {len(pava_groups)}")
print(f"BMI range: [{pava_groups['x'].min():.1f}, {pava_groups['x'].max():.1f}]")

##### 3.2 Binning Results

In [ ]:
# Get binning summary
summary = bmi_binner.summary_()
print("\nBMI Binning Summary:")
print(summary)

# Note: For continuous targets, there's no WoE/IV

In [ ]:
# Comprehensive visualization for continuous target
# Since we don't have WoE for continuous targets, we'll create a custom visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Mean charges by bin
ax1 = axes[0, 0]
plot_event_rate(summary, ax=ax1, title='Mean Charges by BMI Bin', 
                y_format='decimal', show_counts=True)

# Plot 2: Sample distribution
ax2 = axes[0, 1]
plot_sample_distribution(summary, ax=ax2, title='Sample Distribution Across BMI Bins')

# Plot 3: Bin boundaries on data
ax3 = axes[1, 0]
plot_bin_boundaries(bmi_binner, ax=ax3, title='BMI Bin Boundaries', 
                   show_density=True, show_means=False)

# Plot 4: Charges distribution by bin (violin plot)
ax4 = axes[1, 1]
bin_labels = bmi_binner.transform(df['bmi'])
violin_df = pd.DataFrame({'BMI_Bin': bin_labels, 'Charges': df['charges']})
# Filter to numeric bins only
violin_df = violin_df[~violin_df['BMI_Bin'].isin(['Missing', 'Excluded'])]

# Create violin plot
bin_order = summary[~summary['bucket'].str.contains('Missing|Excluded')]['bucket'].tolist()
sns.violinplot(data=violin_df, x='BMI_Bin', y='Charges', order=bin_order, ax=ax4)
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=45, ha='right')
ax4.set_title('Charges Distribution by BMI Bin')

plt.tight_layout()
plt.show()

##### 4. Monotonic Binning: Age with Different Strategies

In [ ]:
# Compare different merge strategies on Age
strategies = [MergeStrategy.HIGHEST_PVALUE, MergeStrategy.SMALLEST_LOSS, MergeStrategy.BALANCED_SIZE]
strategy_results = []

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, strategy in enumerate(strategies):
    # Fit binner with different strategy
    age_binner = MonotonicBinner(
        df=df,
        x='age',
        y='charges',
        constraints=BinningConstraints(max_bins=5, min_samples=0.05),
        merge_strategy=strategy
    )
    age_binner.fit()
    
    # Get results
    summary = age_binner.summary_()
    strategy_results.append({
        'Strategy': strategy.value,
        'N_bins': len(summary[~summary['bucket'].str.contains('Missing|Excluded')]),
        'Min_samples': summary['count'].min(),
        'Max_samples': summary['count'].max(),
        'Std_of_means': summary['mean'].std()
    })
    
    # Plot
    plot_event_rate(summary, ax=axes[idx], 
                   title=f'Age Binning: {strategy.value}',
                   y_format='decimal', show_counts=True)

plt.tight_layout()
plt.show()

# Compare strategies
strategy_df = pd.DataFrame(strategy_results)
print("\nStrategy Comparison:")
print(strategy_df)

##### 5. Advanced: Binning with Log-Transformed Target
Since insurance charges are highly skewed, let's try binning with log-transformed target:

In [ ]:
# Create log-transformed target
df['log_charges'] = np.log1p(df['charges'])

# Bin BMI with log charges
log_binner = MonotonicBinner(
    df=df,
    x='bmi',
    y='log_charges',
    constraints=BinningConstraints(max_bins=5, min_samples=0.05)
)
log_binner.fit()

# Compare with original scale
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Original scale
summary_orig = bmi_binner.summary_()
plot_event_rate(summary_orig, ax=ax1, title='BMI Bins: Original Charges',
               y_format='decimal', show_counts=False)

# Log scale
summary_log = log_binner.summary_()
plot_event_rate(summary_log, ax=ax2, title='BMI Bins: Log(Charges)',
               y_format='decimal', show_counts=False)

plt.tight_layout()
plt.show()

##### 6. Multi-Feature Binning Analysis

In [ ]:
# Bin all numeric features and analyze monotonic patterns
features_to_bin = ['age', 'bmi', 'steps', 'children']
binning_results = []

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(features_to_bin):
    # Fit binner
    binner = MonotonicBinner(
        df=df,
        x=feature,
        y='charges',
        constraints=BinningConstraints(max_bins=5, min_samples=0.05)
    )
    binner.fit()
    
    # Store results
    summary = binner.summary_()
    binning_results.append({
        'Feature': feature,
        'Direction': binner.resolved_sign_,
        'N_bins': len(binner.bins_()),
        'Correlation': df[feature].corr(df['charges']),
        'Mean_range': summary['mean'].max() - summary['mean'].min()
    })
    
    # Plot
    plot_event_rate(summary, ax=axes[idx], 
                   title=f'{feature.capitalize()} ({binner.resolved_sign_})',
                   y_format='decimal', show_counts=True)

plt.tight_layout()
plt.show()

# Display results
results_df = pd.DataFrame(binning_results)
print("\nMulti-Feature Binning Results:")
print(results_df.sort_values('Mean_range', ascending=False))

##### 7. Creating a Binned Dataset for Modeling

In [ ]:
# Transform all features to binned versions
binned_features = {}

for feature in features_to_bin:
    # Fit binner
    binner = MonotonicBinner(
        df=df,
        x=feature,
        y='charges',
        constraints=BinningConstraints(max_bins=5, min_samples=0.05)
    )
    binner.fit()
    
    # Create binned versions
    binned_features[f'{feature}_bin'] = binner.transform(df[feature])
    binned_features[f'{feature}_left'] = binner.transform(df[feature], assign='left')
    binned_features[f'{feature}_right'] = binner.transform(df[feature], assign='right')

# Create new dataframe with binned features
df_binned = pd.DataFrame(binned_features)
print("Binned features dataset:")
print(df_binned.head(10))

##### 8. Stability Analysis with Cross-Validation

In [ ]:
from sklearn.model_selection import KFold

# Perform k-fold stability analysis
kf = KFold(n_splits=5, shuffle=True, random_state=42)
stability_results = []

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]
    
    # Fit on train
    cv_binner = MonotonicBinner(
        df=train_df,
        x='bmi',
        y='charges',
        constraints=BinningConstraints(max_bins=5, min_samples=0.05)
    )
    cv_binner.fit()
    
    # Evaluate on test
    train_summary = cv_binner.summary_()
    
    # Calculate test statistics
    test_bins = cv_binner.transform(test_df['bmi'])
    for bin_label in train_summary['bucket']:
        if not bin_label.startswith('Missing'):
            mask = test_bins == bin_label
            if mask.sum() > 0:
                stability_results.append({
                    'Fold': fold + 1,
                    'Bin': bin_label,
                    'Train_mean': train_summary[train_summary['bucket'] == bin_label]['mean'].iloc[0],
                    'Test_mean': test_df.loc[mask, 'charges'].mean(),
                    'Test_count': mask.sum()
                })

# Analyze stability
stability_df = pd.DataFrame(stability_results)
stability_summary = stability_df.groupby('Bin').agg({
    'Train_mean': 'mean',
    'Test_mean': 'mean',
    'Test_count': 'mean'
}).reset_index()

stability_summary['Diff_%'] = abs(stability_summary['Test_mean'] - stability_summary['Train_mean']) / stability_summary['Train_mean'] * 100

print("\nCross-Validation Stability Analysis:")
print(stability_summary)
print(f"\nAverage difference: {stability_summary['Diff_%'].mean():.2f}%")

##### 9. Detailed PAVA Visualization

In [ ]:
# Deep dive into PAVA for steps feature
steps_binner = MonotonicBinner(
    df=df,
    x='steps',
    y='charges',
    constraints=BinningConstraints(max_bins=6, min_samples=0.05)
)
steps_binner.fit()

# Get PAVA components
groups = steps_binner.pava_groups_()
blocks = steps_binner.pava_blocks_(as_dict=True)

# Create detailed PAVA visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# CSD plot
plot_csd(groups, blocks, ax=ax1, title='Cumulative Sum Diagram: Steps vs Charges')

# GCM plot
plot_gcm(groups, blocks, ax=ax2, title='Greatest Convex Minorant: Steps', 
         show_violations=True)

plt.tight_layout()
plt.show()

print(f"\nPAVA compression: {len(groups)} groups → {len(blocks)} blocks")
print(f"Monotonicity: {steps_binner.resolved_sign_}")

##### 10. Key Insights and Recommendations

In [ ]:
# Get comprehensive diagnostics for all features
print("Comprehensive Binning Analysis:\n")

for feature in features_to_bin:
    binner = MonotonicBinner(
        df=df,
        x=feature,
        y='charges',
        constraints=BinningConstraints(max_bins=5, min_samples=0.05)
    )
    binner.fit()
    
    diag = binner.get_diagnostics()
    print(f"{feature.upper()}:")
    print(f"  Monotonicity: {diag['resolved_sign']}")
    print(f"  PAVA compression: {diag['pava_diagnostics']['compression_ratio']:.2f}x")
    print(f"  Final bins: {diag['n_final_bins']}")
    print(f"  Constraints satisfied: {diag['constraints_satisfied']}")
    print()